<a href="https://colab.research.google.com/github/YunshuoTian/nlp/blob/main/vector_search_airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pymongo
import requests
from google.colab import userdata
import time

mongo_secret = userdata.get('mongo_secret')
client = pymongo.MongoClient(mongo_secret)
hf_token = userdata.get('hf_read')

In [ ]:
db = client.sample_airbnb
collection = db.listingsAndReviews
collection.find_one({'summary':{"$exists": True}})

In [14]:
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text: str) -> list[float]:

    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {hf_token}"},
        json={"inputs": text})

    if response.status_code == 503:
        time.sleep(20)
    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")

    return response.json()

In [19]:
for doc in collection.find({'summary':{"$exists": True}, 'summary_embedding': {'$exists': False}}).limit(100):
	doc['summary_embedding'] = generate_embedding(doc['summary'])
	collection.replace_one({'_id': doc['_id']}, doc)

In [20]:
collection.count_documents({'summary_embedding': {'$exists': True}})

500

In [25]:
query = "Apartment in city center of new york"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "summary_embedding",
    "numCandidates": 100,
    "limit": 4,
    "index": "summary_index",
      }}
])

In [26]:
for document in results:
    print(f'Name: {document["name"]},\nSummary: {document["summary"]},\nSpace: {document["space"]}\n')

Name: UWS Brownstone Near Central Park,
Summary: This spacious one bedroom apartment has a bonus space with two extra beds as well as a fold out sofa.  Close to the park, the subway, museums and restaurants.  This apartment is the perfect quiet getaway in the middle of Manhattan.,
Space: This charming one-bedroom apartment in a landmark brownstone townhouse in the Upper West Side area of Manhattan has high-ceilings, a fireplace and two planted, south-facing balconies overlooking a beautiful garden.  (The balcony off of the living room is about 5" X 8" and has a small table and two chairs where two people can have a nice meal.  The balcony off of the bedroom is about 5" X 14' and there is a chaise longue and a sitting chair, along with two side tables.  Both have outdoor lights, overlook a nice garden, and have plantings.)   There is a queen-sized bed in the private bedroom with closing door. There is also a very high quality full sized sofa bed in the living room.  This beautiful home 